In [53]:
# -*- coding: utf8 -*-
#########################IMPORTS############################################
from os.path import expanduser
import codecs, optparse
import re
import random as rd
import sys,os,time
import string
import yaml, YamlDuplicates
import ParFuMor as PFM
from ParFuMor import *
import pickle

In [54]:
from string import translate, maketrans

In [55]:
home = expanduser("~")
annee="22"
def getSerie(nomKalaba):
#     return home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/Kalaba-Project/%s-%s/"%(annee,nomKalaba)
    return home+"/ownCloud/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s-%s/"%(annee,nomKalaba)


prevSerie=getSerie("K2")
nextSerie=getSerie("K1")

nomsPropres=["Agathos","Clemencia","Elphaba","Freja","Hoderi","Kaleb","Mahira","Violette","Nabil","Katisha","Nicole"]
nomsPropresP=["agaTos","klemensja","elfaba","freZa","oderi","kaleb","maira","vjolet","nabil","katiSa","nikol"]

In [56]:
with open(prevSerie+"Stems.yaml", 'r') as stream:
    prevStems=yaml.safe_load(stream)
with open(nextSerie+"Stems.yaml", 'r') as stream:
    nextStems=yaml.safe_load(stream)

In [57]:
def getLeaves(stemStructure):
    formes={}
    for element in stemStructure:
        if type(stemStructure[element])==list:
            formesCle=", ".join(stemStructure[element])
            formes[formesCle]=element
        else:
            formes.update(getLeaves(stemStructure[element]))
    return formes

In [58]:
prevLexique=getLeaves(prevStems)
nextLexique=getLeaves(nextStems)

In [59]:
def replace_keys(old_dict, key_dict):
    new_dict = { }
    for key in old_dict.keys():
        new_key = key_dict.get(key, key)
        if isinstance(old_dict[key], dict):
            new_dict[new_key] = replace_keys(old_dict[key], key_dict)
        else:
            new_dict[new_key] = old_dict[key]
    return new_dict


In [60]:
with open(prevSerie+"Phonology.yaml", 'r') as stream:
    phonology=yaml.safe_load(stream)

In [61]:
obstrusivesSourdes=u"ptkfTsS"
obstrusivesVoisees=u"bdgvDzZ"

voisement=maketrans(obstrusivesSourdes,obstrusivesVoisees)
devoisement=maketrans(obstrusivesVoisees,obstrusivesSourdes)


consonnes=phonology["consonnes"]
voyelles=phonology["voyelles"]

In [62]:
def devoiserFinale(radical):
    if radical[-1] in obstrusivesVoisees:
        return radical.translate(devoisement)
    else:
        return None

def voiserVCV(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical[1:-1]):
        if l in obstrusivesSourdes and radical[p] in voyelles and radical[p+2] in voyelles:
            cibles.append(p+1)
    if cibles:
        cible=rd.choice(cibles)
        result[cible]=result[cible].translate(voisement)
        return "".join(result)
    else:
        return None

def metathese(radical):
    result=list(radical)
    cibles=[]
    for p,l in enumerate(radical):
        if l in consonnes:
            cibles.append(p)
    if len(cibles)>1 and rd.randint(1,3)==1:
        cible1=rd.choice(cibles[:-1])
        cible2=cibles[cibles.index(cible1)+1]
        result[cible1]=radical[cible2]
        result[cible2]=radical[cible1]
        result="".join(result)
        if result!=radical:
            return result
        else:
            return None
    else:
        return None

    
def apophonie(radical,choice="rdVowel"):
    result=list(radical)
    cibles=[]
    if not choice:
        choice=rd.choice(["apophonie","rdVowel"])
    if choice=="apophonie":
        for p,l in enumerate(radical):
            if l in "iae":
                cibles.append(p)
        if cibles:
            cible=rd.choice(cibles)
            result[cible]=phonology["apophonies"][result[cible]]
            return "".join(result)
        else:
            return None
    else:
        for p,l in enumerate(radical):
            if l in voyelles:
                cibles.append(p)
        if cibles:
            cible=rd.choice(cibles)
            result[cible]=rd.choice(voyelles)
            return "".join(result)
        else:
            return None

def changerLongueur(radical):
    result=list(radical)
    if len(radical)==5:
        if radical[0] in consonnes:
            if radical[-1] in consonnes:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(voyelles))
                    return "".join(result)
                else:
                    return None
            else:
                if rd.randint(1,5)==1:
                    result.append(rd.choice(consonnes))
                    return "".join(result)
                else:
                    return None
        else:
            if rd.randint(1,3)==1:
                result.insert(0,rd.choice(consonnes))
                return "".join(result)
            else:
                return None
    elif len(radical)>5:
        if rd.randint(1,3)==1:
            if radical[0] in consonnes:
                result=result[:-1]
            else:
                result=result[1:]
            return "".join(result)
        else:
            return None
    else:
        if rd.randint(1,3)==1:
            if radical[0] in voyelles:
                result.insert(0,rd.choice(consonnes))
            elif radical[-1] in voyelles:
                result.append(rd.choice(consonnes))
            else:
                result.append(rd.choice(voyelles))
            return "".join(result)
        else:
            return None
        
            

In [63]:
apophonie("kulubu",choice="rnd")

'kalubu'

In [64]:
operations=[
    devoiserFinale,
    apophonie,apophonie,
    voiserVCV,voiserVCV,
    metathese,
    changerLongueur
           ]

In [65]:
def modifierRadical(radical):
    nOperations=rd.choice([0,1,1,1,2,2,2,3,3,3,4,5])
    while nOperations>0:
        operation=rd.choice(operations)
        result=operation(radical)
        if result:
            radical=result
            nOperations-=1
    return radical

In [66]:
racineChanges={}
prevVedettes={m.split(",")[0]:m for m in prevLexique}
nextVedettes={m.split(",")[0]:m for m in nextLexique}
print "lexique suivant"
for mot in prevLexique:
    if mot in nextLexique:
        if prevLexique[mot] in nomsPropresP:
            racineChanges[nextLexique[mot]]=prevLexique[mot]
        else:
            racineChanges[nextLexique[mot]]=modifierRadical(prevLexique[mot])
#         print prevLexique[mot],racineChanges[nextLexique[mot]]
    else:
        print mot
        vedette=mot.split(",")[0]
        if vedette in nextVedettes:
            print "\t≠",nextVedettes[vedette]
        
print "================="
print "lexique précédent"
for mot in nextLexique:
    if not mot in prevLexique:
        print mot
        vedette=mot.split(",")[0]
        if vedette in prevVedettes:
            print "\t≠",prevVedettes[vedette]




lexique suivant
villageois.HYPER, villagEOIs, villagEOIS
enfant, enfants, garçon, garçons
	≠ enfant, enfants, enfANT, enfANTs
villageois, villageoiS
	≠ villageois, villageoiS, villagEOIs, villagEOIS
enfant.HYPER, enfANT, enfANTs
chasseur, chasseurs, chasseuse, chasseuses
	≠ chasseur, chasseurs
lexique précédent
chasseuse, chasseuses
garçon, garçons
chasseur, chasseurs
	≠ chasseur, chasseurs, chasseuse, chasseuses
villageois, villageoiS, villagEOIs, villagEOIS
	≠ villageois, villageoiS
enfant, enfants, enfANT, enfANTs
	≠ enfant, enfants, garçon, garçons


In [67]:
for i in range(10):
    print modifierRadical("Tonif")

Tinif
Tenif
Tunof
Toneva
Tenif
Tifin
Tonif
Tonif
Tonif
Tenif


In [68]:
racineChanges

{'DaZ': 'kiDe',
 'DagiN': 'tedut',
 'DeZal': 'duZuS',
 'DuDiT': 'fabik',
 'DuDul': 'tibizi',
 'Duf': 'pol',
 'Nabet': 'kiZik',
 'Nalok': 'NiZiN',
 'Namut': 'ZiDaN',
 'Nineta': 'ditiT',
 'NoSoT': 'TuNam',
 'Nob': 'Sup',
 'NovoT': 'Tepap',
 'Nubof': 'kages',
 'Sagit': 'rekaS',
 'Sareg': 'torip',
 'Sazep': 'bedaze',
 'Segop': 'weSove',
 'Seris': 'mesaz',
 'Sibop': 'ZeZeDi',
 'Sidubi': 'NoZube',
 'Sol': 'las',
 'Sosak': 'balaz',
 'Spok': 'kabaTe',
 'SuS': 'bame',
 'SugeD': 'riguN',
 'TaZar': 'TuZir',
 'TaraZ': 'nokol',
 'Tarit': 'Zonar',
 'TeniS': 'voZog',
 'Tesok': 'luZowi',
 'Tibaf': 'Dodum',
 'TimeZ': 'Tozop',
 'Timok': 'teDavu',
 'Titem': 'nazeku',
 'Togid': 'maDoS',
 'Tosapi': 'roziw',
 'Zavari': 'logam',
 'ZeZij': 'Nidop',
 'Zefan': 'teZas',
 'Zegun': 'bedibi',
 'Zidik': 'puZas',
 'ZizuZ': 'Tebos',
 'Zorik': 'libez',
 'agaTos': 'agaTos',
 'b': 'pa',
 'bagak': 'vibonu',
 'bageN': 'bogeNe',
 'baged': 'Nigudu',
 'bavod': 'ronaga',
 'beNep': 'fivok',
 'beZiT': 'mijete',
 'begut': 'nales'

In [69]:
newStems=replace_keys(nextStems,racineChanges)

In [70]:
with open(nextSerie+'flowStems.yaml', 'w') as output:
    yaml.dump(newStems, output, default_flow_style=False,allow_unicode=True)

In [71]:
with open(nextSerie+"flowStems.yaml", 'r') as input:
    yamlLines=input.readlines()

In [72]:
yamlText="".join(yamlLines)
yamlText=re.sub(r"\n\s*-\s*",", ",yamlText)
yamlText=re.sub(r":,\s*",": ",yamlText)
yamlText=re.sub(r": *([^\n]+)",": [\g<1>]",yamlText)
print yamlText

ADJ:
  DuNaN: [dernier, derniers, dernière, dernières]
  Nebun: [troisième, troisièmes]
  NiZiN: [deux]
  Tabot: [gros, grosse, grosses]
  badoT: [jaune, jaunes]
  beNar: [vilain, vilains, vilaine, vilaines]
  biDuS: [rouge, rouges]
  bogeNe: [héroïque, héroïques]
  felipi: [disparu, disparus, disparue, disparues]
  godes: [profond, profonds, profonde, profondes]
  gokit: [effrayé, effrayés, effrayée, effrayées]
  guZuS: [suivant, suivants, suivante, suivantes]
  gubedo: [blessé, blessés, blessée, blessées]
  jenek: [différent, différents, différente, différentes]
  kabaTe: [six]
  leDot: [autre, autres]
  loZaS: [treize]
  luzuNa: [nouveau, nouveaux, nouvelle, nouvelles]
  mebate: [furieux, furieuse, furieuses]
  molig: [quatre]
  nales: [nombreux, nombreuses]
  nitiv: [trois]
  niwodu: [blanc, blancs, blanche, blanches]
  nobuZi: [maigre, maigres]
  nokol: [bas, basse, basses]
  pevos: [noir, noirs, noire, noires]
  pubiT: [méchant, méchants, méchante, méchantes]
  rabeve: [terrible,

In [73]:
with open(nextSerie+'newStems.yaml', 'w') as output:
    output.write(yamlText)

In [74]:
if 'VI' in nextStems['VER'].keys():
    prevVI=set()
    for cf,v in prevStems['VER']['VI'].iteritems():
        for k,l in v.iteritems():
            prevVI.add(l[0])
    nextVI=set()
    for cf,v in nextStems['VER']['VI'].iteritems():
        for k,l in v.iteritems():
            nextVI.add(l[0])

    print prevVI-nextVI
    print nextVI-prevVI
else:
    print "pas de distinction VI/VT"

pas de distinction VI/VT
